<font size=6 bold=true>Feb 16, 2020  Part A
<br><br>Content cleaning</font>  

In [1]:
import re
import spacy 
from spacy import displacy
import en_core_web_sm
from pathlib import Path
from IPython.display import display, Markdown, Latex 
import sys
import csv
import pandas as pd
import numpy as np
import os
from os import path, listdir
from urllib.parse import urlparse, urljoin
import requests
import ntpath
from bs4 import BeautifulSoup, Comment
import time
from pprint import pprint 
import shutil
#!pip install tldextract
import tldextract
from pathlib import Path
from datetime import datetime
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_colwidth', -1)
 

In [11]:
class Get_Url_content():
    def __init__(self, 
                 folder='C:\\Users\\Kentt\\stempro\\hw\\y2020feb16\\learning_data', 
                 result_folder='C:\\Users\\Kentt\\stempro\\hw\\y2020feb16\\learning_data\\result',
                 data_pattern='^A_.+\d+.csv$', 
                 url_file_pattern='A_.+\d+url.csv$',
                 college_list_file='C:\\Users\\Kentt\\stempro\\hw\\y2020feb16\\learning_data\\ContentAqusition__All.csv', 
                 data_sum_file='learning_data_sum.csv', 
                 url_sum_file='learning_url_sum.csv',
                 data_info_file='learning_data_info.csv'
                ):
        self.folder=folder
        self.result_folder=result_folder
        self.data_pattern=data_pattern
        self.url_file_pattern=url_file_pattern
        self.college_list_file=college_list_file
        self.data_sum_file=data_sum_file
        self.url_sum_file=url_sum_file
        self.data_info_file=data_info_file
    def Get_Files(self, pattern):
        return [[os.stat(path.join(self.folder, f)).st_size, path.join(self.folder, f), f] 
                for f in listdir(self.folder) if re.match(pattern, f)]
    def Get_data_info(self): 
        #data files
        #curl=Get_Url_content('C:/Stempro/Notebooks/data')
        data_files=self.Get_Files(self.data_pattern)
        data_files.sort(key = lambda x: x[0],reverse=True) 
        #to df
        df_files=pd.DataFrame({'size_bytes':[c1[0]/1000.0 for c1 in data_files], 'full_path':[c1[1] for c1 in data_files], 'basename':[c1[2] for c1 in data_files]})
        df_files['accumu_size']=df_files['size_bytes'].apply('cumsum')
        df_files['per']=100.0*df_files['accumu_size']/df_files['size_bytes'].apply('sum')
        
        #url files 
        url_files=self.Get_Files(self.url_file_pattern)
        df_url_files=pd.DataFrame({'size_bytes':[c1[0] for c1 in url_files], 'full_path':[c1[1] for c1 in url_files], 'basename':[c1[2] for c1 in url_files]})
        
        merge_file_stats=pd.concat([pd.read_csv(i[1][1], sep='\t').groupby('status_code')['url'].count().reset_index().assign(college=i[1][2]) for 
        i in df_url_files.iterrows()], axis=0)  
        
        df_stats=merge_file_stats.groupby('college').apply(self.Agg_college).reset_index()
         
        #master file df 
        df_master=pd.read_csv(self.college_list_file, sep=',') 
        
        df_master=df_master[['name', 'url', 'filename', 'urlfile']].merge(df_stats[['college', 'total_pages','visited', 'rejected']], 
            left_on='urlfile', right_on='college', how='left')
        df_master=df_master.merge(df_files[['basename', 'size_bytes', 'accumu_size', 'per']], left_on='filename', 
                              right_on='basename', how='left')
        df_master.fillna('', inplace=True) 
        df_master.to_csv(path.join(self.result_folder, self.data_info_file), sep='\t', index=False) 
        return df_master.head(100)
    
    def Assign_url_id(self):
        datafiles=self.Get_Files(self.data_pattern)
        huge=pd.concat([pd.read_csv(i[1], sep='\t')[['url', 'text']].groupby('url')['text'].count().reset_index().assign(college=i[2]) 
                        for i in datafiles], axis=0)  
        huge.reset_index(inplace=True)
        df_master=pd.read_csv(self.college_list_file, sep=',') 
        #df_master=pd.read_csv( 'C:/Stempro/Notebooks/training/ContentAqusition__All.csv', sep=',') 
        huge=huge.merge(df_master[['collegeId', 'url', 'name', 'filename']], 
                        left_on='college', right_on='filename', how='right')

        huge.rename(columns={'index': 'url_id', 'text': 'lines', 'url_x':'url', 'url_y': 'college_url', 'college':'data_file'}, inplace=True) 
        huge.drop(columns=['filename'], inplace=True) 
        return huge 
    
    def Clean_one_file(self, file):
        return pd.read_csv(self.college_list_file, sep=',')
    
    def Agg_college(self,x):
        d = {}
        d['total_pages'] = x['url'].sum() 
        d['visited'] = x[x['status_code']==200]['url'].sum() 
        d['rejected'] = x[(x['status_code']!=200) & (x['status_code']!=0)]['url'].sum() 
        return pd.Series(d, index=['total_pages', 'visited', 'rejected'])
    def Agg_url(self,x): 
        d = {}
        d['entries'] = x['text'].count() 
        d['textlen'] = x['len'].sum()  
        return pd.Series(d, index=['entries', 'textlen'])
    
    def Merge_url_and_data(self):
        datafiles=self.Get_Files(self.data_pattern)
        df_url_sum=pd.DataFrame()
        df_data_sum=pd.DataFrame() 
        for i in datafiles:
            try:
                file=i[1]
                college_id=int(''.join(j for j in i[2] if j.isdigit()))     
                dfdata=pd.read_csv(file, sep='\t') 
                dfdata.fillna('', inplace=True) 
                dfdata['text']=dfdata['text'].astype(str)
                dfdata['len']=dfdata['text'].apply((lambda x: len(x)))
                dfdata=dfdata[~dfdata['url'].str.contains('.pdf', case=False)]
                
                df_urls=dfdata.groupby('url').apply(self.Agg_url).reset_index().reset_index() 
                df_urls.rename(columns={'index':'id_url'}, inplace=True)
                df_urls=df_urls.assign(id_college=college_id)
                df_url_sum=pd.concat([df_url_sum, df_urls])

                df_data_new=dfdata[['url', 'parent', 'ps', 'ns', 'text']].merge(df_urls, left_on='url', right_on='url', how='inner')
                df_data_new.drop(columns=['url', 'textlen', 'entries'], inplace=True) 

                df_data_sum=pd.concat([df_data_sum, df_data_new])
            except Exception as e:
                print(e, file)
        df_url_sum.to_csv(path.join(self.result_folder, self.url_sum_file), sep='\t', index=False)
        df_data_sum.to_csv(path.join(self.result_folder, self.data_sum_file), sep='\t', index=False) 
        return df_url_sum.head(100), df_data_sum.head(100)

""" 
ab=Get_Url_content()
u,d=ab.Merge_url_and_data() 
""" 

' \nab=Get_Url_content()\nu,d=ab.Merge_url_and_data() \n'

In [12]:
ab=Get_Url_content()
ab.Get_data_info()
ab.Merge_url_and_data()

(    id_url  \
 0   0        
 1   1        
 2   2        
 3   3        
 4   4        
 .. ..        
 95  95       
 96  96       
 97  97       
 98  98       
 99  99       
 
                                                                        url  \
 0   http://admissionblog.agnesscott.org/category/decisions                   
 1   http://admissionblog.agnesscott.org/category/journeys                    
 2   http://agnesscott.edu/admission/high-school-students/how-to-apply.html   
 3   http://agnesscott.edu/admission/index.html                               
 4   http://agnesscott.edu/visit                                              
 ..                          ...                                              
 95  http://jobs.atsu.edu                                                     
 96  http://jobs.atsu.edu/about-atsu                                          
 97  http://jobs.atsu.edu/afainstitute                                        
 98  http://jobs.atsu.edu/ah